In [1]:
import opendatasets as od 
od.download("https://www.kaggle.com/datasets/umitka/twitter-toxic-tweets")

Skipping, found downloaded files in ".\twitter-toxic-tweets" (use force=True to force download)


In [2]:
import pandas as pd
import numpy as np 
import nltk
import re
nltk.download('all')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\noman\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\noman\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\noman\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\noman\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\noman\AppData\Roaming\nltk_data...
[

In [3]:
df = pd.read_csv("twitter-toxic-tweets/twitter_toxic_tweets.csv",index_col="id")

In [4]:
df.head()

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation


In [5]:
df["tweet"]

id
1         @user when a father is dysfunctional and is s...
2        @user @user thanks for #lyft credit i can't us...
3                                      bihday your majesty
4        #model   i love u take with u all the time in ...
5                   factsguide: society now    #motivation
                               ...                        
31958    ate @user isz that youuu?ðððððð...
31959      to see nina turner on the airwaves trying to...
31960    listening to sad songs on a monday morning otw...
31961    @user #sikh #temple vandalised in in #calgary,...
31962                     thank you @user for you follow  
Name: tweet, Length: 31962, dtype: object

In [6]:
def preprocess(text):
    #lowercasing
    text = text.lower()
    
    #removing urls
    text = re.sub(r"https?://\S+|www\.\S+","",text)
    
    #removing mentions
    text = re.sub(r"@\w+", "", text)
    
    #removing hastag symbols
    text = re.sub(r"#", "", text)
    
    #remove numbers
    text = re.sub(r"\d+", "",text)
    
    #remove special characters
    text = re.sub(r"[^\w\s]", "",text)
    
    #remove extra spaces
    text = re.sub(r"\s+"," ", text).strip()
    
    #tokenize
    words= word_tokenize(text)
    
    #stop words removal
    words = [word for word in words if word not in stop_words]
    
    #lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return " ".join(words)

In [9]:
df["cleaned_tweets"] = df["tweet"].apply(preprocess)

In [10]:
df.head()

,label,tweet,cleaned_tweets
id,,,
1,0,@user when a father is dysfunctional and is s...,father dysfunctional selfish drag kid dysfunct...
2,0,@user @user thanks for #lyft credit i can't us...,thanks lyft credit cant use cause dont offer w...
3,0,bihday your majesty,bihday majesty
4,0,#model i love u take with u all the time in ...,model love u take u time urð ðððð ððð
5,0,factsguide: society now #motivation,factsguide society motivation


In [11]:
X = df["cleaned_tweets"]
y = df["label"]

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((25569,), (6393,), (25569,), (6393,))

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [16]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,3))),
    ("lr", LogisticRegression(max_iter=2000, penalty="l2", C=1.0))
])

In [17]:
pipeline.fit(X_train, y_train)

c:\Users\noman\anaconda3\envs\mlai\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('lr', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [19]:
y_pred = pipeline.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.9429063037697482


In [21]:
print("Classification Report: 0 means non toxic tweets and 1 is toxic tweets \n ", classification_report(y_test, y_pred))

Classification Report: 0 means non toxic tweets and 1 is toxic tweets 
                precision    recall  f1-score   support

           0       0.94      1.00      0.97      5937
           1       0.86      0.24      0.37       456

    accuracy                           0.94      6393
   macro avg       0.90      0.62      0.67      6393
weighted avg       0.94      0.94      0.93      6393



In [31]:
#using naive bayes
pipeline1 = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(2,4))),
    ("mnv", MultinomialNB())
])

In [32]:
pipeline1.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('mnv', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [33]:
y_pred=pipeline1.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))

Accuracy:  0.9382136712028781


In [34]:
print("Classification Report: 0 means non toxic tweets and 1 is toxic tweets \n ", classification_report(y_test, y_pred))

Classification Report: 0 means non toxic tweets and 1 is toxic tweets 
                precision    recall  f1-score   support

           0       0.94      1.00      0.97      5937
           1       1.00      0.13      0.24       456

    accuracy                           0.94      6393
   macro avg       0.97      0.57      0.60      6393
weighted avg       0.94      0.94      0.92      6393

